# Combining datasets

### Here we work towards a method to combine datasets and outputs predictions

#### We will use: SPROF Predictions, ProFun (https://github.com/SamusRam/ProFun) Predictions and QuickGo Annotations

In [ ]:
#Create a dictionary to assign each go term to the roots (CCO, MFO, BPO)

['/kaggle/input/proteinet-best/blast_submission.tsv',
         '/kaggle/input/quick-go-2022-03-02/quickgo.tsv',
         '/kaggle/input/mycopy-oleg-5877/submission.tsv',
         '/kaggle/input/mycopy-oleg-5877/submission.tsv',
         '/kaggle/input/push2-577/submission.tsv',
         '/kaggle/input/mine-57253/submission.tsv',
         '/kaggle/input/mine-57167/submission_mine57167.tsv',
         '/kaggle/input/sidh-55982/submission.tsv',
         '/kaggle/input/zamus-56916/submission_zamus_56916.tsv',
         /kaggle/input/sub-ker-5580/submission.tsv
         /kaggle/input/sub-ker-560/submission.tsv
         /kaggle/input/sub-tor-0529/submission_t0529.tsv
         /kaggle/input/sub-tor-52579/submission.tsv
         
         
        ]


['/kaggle/input/proteinet-best/blast_submission.tsv',
         '/kaggle/input/quick-go-2022-03-02/quickgo.tsv',
         '/kaggle/input/mycopy-oleg-5877/submission.tsv',
         '/kaggle/input/mycopy-oleg-5877/submission.tsv',
         '/kaggle/input/push2-577/submission.tsv',
         '/kaggle/input/mine-57253/submission.tsv',
         '/kaggle/input/mine-57167/submission_mine57167.tsv',
         '/kaggle/input/sidh-55982/submission.tsv',
         '/kaggle/input/zamus-56916/submission_zamus_56916.tsv',
         '/kaggle/input/sub-ker-5580/submission.tsv',
         '/kaggle/input/sub-ker-560/submission.tsv',
         '/kaggle/input/sub-tor-0529/submission_t0529.tsv',
         '/kaggle/input/sub-tor-52579/submission.tsv'                 
        ]

In [3]:
import polars as pl
import gc

files = ['/kaggle/input/mycopy-oleg-5877/submission.tsv',
         '/kaggle/input/mycopy-oleg-5877/submission.tsv',
         '/kaggle/input/push2-577/submission.tsv',
         '/kaggle/input/mine-57253/submission.tsv',
         '/kaggle/input/mine-57167/submission_mine57167.tsv',
         '/kaggle/input/sidh-55982/submission.tsv',
         '/kaggle/input/zamus-56916/submission_zamus_56916.tsv',
         '/kaggle/input/sub-ker-5580/submission.tsv',
         '/kaggle/input/sub-ker-560/submission.tsv',
         '/kaggle/input/sub-tor-0529/submission_t0529.tsv',
         '/kaggle/input/sub-tor-52579/submission.tsv']

keys = set()
for file_name in files:
    file = pl.read_csv(file_name, separator='\t', has_header=False)
    keys_ = file.with_columns((pl.col('column_1').cast(str) + '|'+pl.col('column_2').cast(str)).alias('key'))['key'].to_list()
    keys = keys.union(set(keys_))
    del file
    gc.collect()
    
acc = pl.Series(list(keys))
acc = acc.rename('key').to_frame().with_columns([pl.lit(0).alias("prob_acc"),
                                                 pl.lit(0).alias("exists")])
for file_name in files:
    file = pl.read_csv(file_name, separator='\t', has_header=False)
    if len(file.columns) == 3:
        k1 = 'column_1'
        k2 = 'column_2'
        v =  'column_3'
    else:
        k1 = 'column_2'
        k2 = 'column_3'
        v =  'column_4'
    file = file.with_columns((pl.col(k1).cast(str) + '|'+pl.col(k2).cast(str)).alias('key'))
    file = file.with_columns(pl.col(v).is_not_null().alias('exists').cast(int))
    val = file.filter(pl.col('key') == 'P53566|GO:0009892')
    print(val)
    acc = acc.join(file[[v, 'key', 'exists']], 
                   on = 'key',
                   how = 'left',
                   suffix = 'temp').with_columns([(pl.col(v).fill_null(0) + pl.col('prob_acc')).alias('prob_acc'),
                       (pl.col('existstemp').fill_null(0) + pl.col('exists')).alias('exists'),
                      ])[['prob_acc', 'key', 'exists']]
    del file
    gc.collect()
    


shape: (0, 5)
┌──────────┬──────────┬──────────┬─────┬────────┐
│ column_1 ┆ column_2 ┆ column_3 ┆ key ┆ exists │
│ ---      ┆ ---      ┆ ---      ┆ --- ┆ ---    │
│ str      ┆ str      ┆ f64      ┆ str ┆ i64    │
╞══════════╪══════════╪══════════╪═════╪════════╡
└──────────┴──────────┴──────────┴─────┴────────┘
shape: (0, 5)
┌──────────┬──────────┬──────────┬─────┬────────┐
│ column_1 ┆ column_2 ┆ column_3 ┆ key ┆ exists │
│ ---      ┆ ---      ┆ ---      ┆ --- ┆ ---    │
│ str      ┆ str      ┆ f64      ┆ str ┆ i64    │
╞══════════╪══════════╪══════════╪═════╪════════╡
└──────────┴──────────┴──────────┴─────┴────────┘


In [4]:
acc = acc.filter(pl.col('prob_acc') != 0)

In [5]:
res = acc.with_columns([
    (pl.col('prob_acc') / pl.col('exists')).alias('probs'),
    pl.col('key').str.replace("\|.+", "").alias('key1'),
    pl.col('key').str.replace(".+\|", "").alias('key2'),
])
res = res.sorted(by = key1)

In [ ]:
quik_go = pl.read_csv('/kaggle/input/quick-go-2022-03-02/quickgo.tsv', separator='\t', has_header=True)

['/kaggle/input/mycopy-oleg-5877/submission.tsv', '/kaggle/input/mycopy-oleg-5877/submission.tsv', '/kaggle/input/push2-577/submission.tsv', '/kaggle/input/mine-57253/submission.tsv', '/kaggle/input/mine-57167/submission_mine57167.tsv', '/kaggle/input/sidh-55982/submission.tsv', '/kaggle/input/zamus-56916/submission_zamus_56916.tsv', '/kaggle/input/sub-ker-5580/submission.tsv', '/kaggle/input/sub-ker-560/submission.tsv', '/kaggle/input/sub-tor-0529/submission_t0529.tsv', '/kaggle/input/sub-tor-52579/submission.tsv'
]

In [39]:
!head /kaggle/input/quick-go-2022-03-02/quickgo.tsv

	uniprot	term
0	Q3UPK1	GO:0042393
1	Q3UPK1	GO:0006357
2	Q3UPK1	GO:0005634
3	Q3UPK1	GO:0031491
4	Q3UPK1	GO:0046872
5	Q3UPK1	GO:0042393
6	Q3UPK1	GO:0031491
7	Q3UPK1	GO:0003014
8	Q3UPK1	GO:0006325


In [38]:
#submission = res[['key1', 'key2', 'probs']].sort_values(by = 'key1')
#res[['key1', 'key2', 'probs']].sort(by = 'key1')


key1,key2,probs
str,str,f64
"""A0A009IHW8""","""GO:0003953""",1.0
"""A0A009IHW8""","""GO:0061809""",1.0
"""A0A009IHW8""","""GO:0007165""",1.0
"""A0A009IHW8""","""GO:0016787""",1.0
"""A0A009IHW8""","""GO:0019677""",1.0
"""A0A009IHW8""","""GO:0050135""",1.0
"""A0A021WW32""","""GO:0070193""",1.0
"""A0A021WW32""","""GO:0000795""",1.0
"""A0A021WW32""","""GO:0010628""",1.0


In [ ]:
res[['key1', 'key2', 'probs']].write_csv('submission.tsv', separator='\t', has_header=False)

In [34]:
##res.columns

['prob_acc', 'key', 'exists', 'probs', 'key1', 'key2']